## Normal estimation
### Numpy
ここでは、座標値のみを持つ点の点群から法線を推定します。  

法線の推定は、[PCLのEstimating Surface Normals in a PointCloud](https://pcl.readthedocs.io/projects/tutorials/en/pcl-1.12.0-rc1/normal_estimation.html)を参考に作成します。

$$
\mathcal{C}=\frac{1}{k} \sum_{i=1}^{k} \cdot\left(\boldsymbol{p}_{i}-\overline{\boldsymbol{p}}\right) \cdot\left(\boldsymbol{p}_{i}-\overline{\boldsymbol{p}}\right)^{T}, \mathcal{C} \cdot \overrightarrow{\mathbf{v}}_{j}=\lambda_{j} \cdot \overrightarrow{\mathbf{v}_{j}}, j \in\{0,1,2\}
$$

このsubsectionでは以下のパッケージを使用します。


In [ ]:
import numpy as np
import k3d
import os
from tutlibs.operation import grouping, normal_estimation
from tutlibs.io import Points as io
from tutlibs.visualization import Points as visualizer

推定は以下のとおりです。

In [ ]:
xyz, _, _ = io.read('../data/bunny_pc.ply')
colors = np.tile(np.array([[0, 0, 1]]), (len(xyz), 1))

normals = normal_estimation(xyz)

normal_action = [0]
if 0 in normal_action:
    plot = k3d.plot()
    line_coords = []
    start_index = 0
    end_index = 30
    for normal_index in list(range(len(xyz)))[start_index:end_index]:
        plot += k3d.line([xyz[normal_index], xyz[normal_index]+normals[normal_index]*0.1], width=0.0010, color=0xff0000)
    # visualizer.k3d(xyz, plot=plot)
    normal_knn_points = knn_points[start_index:end_index]
    normal_knn_points = normal_knn_points.reshape(len(normal_knn_points)*k, 3)
    normal_knn_colors = np.tile(np.array([[0, 1, 1]]), (len(normal_knn_points), 1))

    visualizer.k3d(np.vstack([xyz, normal_knn_points]),
                   np.vstack([colors, normal_knn_colors]), plot=plot)

if 1 in normal_action:
    normals_colors = normals
    visualizer.k3d(xyz, normals_colors, color_range=[-1, 1])

if 2 in normal_action:
    os.makedirs('outputs/', exist_ok=True)
    io.write('outputs/numpy_normal.ply', xyz, normals_colors)


Output()

### Use of Open3D
ここで使用するパッケージは以下のとおりです。

In [ ]:
pc = o3d.io.read_point_cloud('../data/bunny_pc.ply')
pc.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
normals = np.asarray(pc.normals)

normals_colors = (normals + 1) / 2
xyz = np.asarray(pc.points)
visualizer.k3d(xyz, normals_colors)
normals_colors = normals_colors * 255

os.makedirs('outputs/', exist_ok=True)
io.write('outputs/o3d_normal.ply', xyz, normals_colors)

Output()